In [1]:
import sys
sys.path.append('..')
import jackknife as jk

import pandas as pd
import numpy as np

## Load Dataset

In [2]:
tmp = pd.read_table("../data/demo-data-1.txt", delim_whitespace=True, header=None)
df = tmp[[4,7,8,15]]
df.columns=["pop", "phys", "beds", "income"]
df.head()

,pop,phys,beds,income
0,8863164,23677,27700,184230
1,5105067,15153,21550,110928
2,2818199,7553,12449,55003
3,2498016,5905,6179,48931
4,2410556,6062,6369,58818


## Specify the Model/Algorithm

In [3]:
def linreg_ols_lu(y, X):
    import numpy as np
    try:  # solve OLS formula
        return np.linalg.solve(np.dot(X.T, X), np.dot(X.T, y))
    except np.linalg.LinAlgError:
        print("LinAlgError: X*X' is singular or not square.")
        return None

In [4]:
def myfunc(data):
    return linreg_ols_lu( data[:,0], data[:,1:] );

## Prep the Data - Use log data

In [5]:
y_train = np.log(df['phys'].values)
X_train = np.c_[np.ones(shape=(len(y_train),)), np.log(df['beds'].values)]

## Run Jackknife Estimation

In [6]:
theta_subsample, theta_fullsample = jk.jk_loop(
    myfunc, np.c_[y_train, X_train], d=1)

In [7]:
pvalues, tscores, theta_jack, se_jack, theta_biased = jk.jk_stats(
    theta_subsample, theta_fullsample)

In [8]:
jk.jk_print(
    pvalues, tscores, theta_jack, se_jack, theta_biased, theta_fullsample, 
    varnames=['intercept', 'beds'], N=len(y_train), d=1)


Delete-1 Jackknife, N=440
                                 intercept    beds    
                      p-Values:    0.00004    0.00000 
                      t-Scores:   -4.15554   40.79625 
 Jackknife Standard Error (SE):    0.17872    0.02514 
   Jackknife Estimates (theta):   -0.743      1.026   
     Jackknife Biased Estimate:   -0.744      1.026   
          Full Sample Estimate:   -0.744      1.026   


## How does the model actually look like?
Convert parameter estimates to original scale

In [9]:
y_predict = np.exp(np.dot(X_train,theta_jack))
#y_predict

In [10]:
ssr = np.sum((y_predict - df['phys'].values)**2) 
ssr

183645117.08850178

In [11]:
s2 = np.sum(  (df['phys'].values - np.mean(df['phys'].values))**2 )
s2

1406206298.9977272

In [12]:
R2 = 1 - ssr/s2
R2

0.869403858296329

## Conclusion
Using log values can remove parameter instability caused by "outliers". 